In [1]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 96.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 9.6 MB/s eta 0:00:0

In [2]:
import pandas as pd
import torch
import datasets
import transformers
from transformers import AutoTokenizer,AutoModelForMaskedLM
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments
from transformers import Trainer
import warnings
warnings.filterwarnings("ignore")

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
valid_df=pd.read_csv('/content/drive/Shareddrives/Embible/Data/DFs without niqqud/valid_df_no_niqqud.csv', sep='\\t', encoding='utf-8')

In [7]:
print(valid_df.columns.tolist())

['Unnamed: 0', 'name', 'verse_idx', 'verse']


In [8]:
print(valid_df.name.unique())

['Isaiah' 'Judges' 'Nehemiah' 'Leviticus' 'Samuel_2' 'Ezra' 'Samuel_1'
 'Ruth' 'Habakkuk' 'Jonah' 'Jeremiah' 'Obadiah' 'Joel' 'Hosea'
 'Song_of_Songs' 'Micah' 'Lamentations' 'Kings_1' 'Kings_2' 'Esther'
 'Haggai' 'Proverbs' 'Genesis' 'Job' 'Deuteronomy' 'Daniel' 'Numbers'
 'Chronicles_1' 'Ezekiel' 'Zephaniah' 'Psalms' 'Nahum' 'Amos' 'Zechariah'
 'Malachi' 'Ecclesiastes' 'Chronicles_2' 'Joshua' 'Exodus']


In [9]:
Period_dic={'Monarchic':['Hosea','Nahum','Zephaniah','Habakkuk','Isaiah','Judges','Samuel_1','Samuel_2','Kings_1','Kings_2','Micah','Proverbs','Deuteronomy','Psalms','Amos','Joshua'],
            'Exilic':['Obadiah','Lamentations','Ezekiel','Jeremiah'],
            'Post-exilic Persian':['Genesis','Exodus','Leviticus','Numbers','Ruth','Haggai','Zechariah','Malachi','Chronicles_1','Chronicles_2','Ezra','Nehemiah','Joel'],
            'Post-exilic Hellenistic':['Job','Ecclesiastes','Song_of_Songs','Jonah','Esther'],
            'Hasmonean':['Daniel']}
Biblical_Hebrew_dic={'Classical Biblical Hebrew':['Genesis','Exodus','Leviticus','Numbers','Deuteronomy','Joshua','Judges','Samuel_1','Samuel_2','Kings_1','Kings_2','Isaiah'],
                     'Transitional period between the classical language and the later language':['Jeremiah','Ezekiel'],
                     'Late Biblical Hebrew':['Ezra','Nehemiah','Esther','Daniel','Chronicles_1','Chronicles_2','Haggai','Zechariah','Malachi','Song_of_Songs','Ecclesiastes']}

In [10]:
Period_lst=[]
flag=False
for i in range(len(valid_df)):
  for k,v in Period_dic.items():
    if(valid_df['name'][i] in v):
      Period_lst.append(k)
      flag=True
  if(not flag):
    Period_lst.append('TBD')
  flag=False

Biblical_Hebrew_lst=[]
flag=False
for i in range(len(valid_df)):
  for k,v in Biblical_Hebrew_dic.items():
    if(valid_df['name'][i] in v):
      Biblical_Hebrew_lst.append(k)
      flag=True
  if(not flag):
    Biblical_Hebrew_lst.append('TBD')
  flag=False

valid_df['Period']=Period_lst
valid_df['Biblical_Hebrew_era']=Biblical_Hebrew_lst


In [11]:
import plotly.express as px
fig = px.histogram(valid_df, x="Period")
fig.show()

In [12]:
import plotly.express as px
fig = px.histogram(valid_df, x="Biblical_Hebrew_era")
fig.show()

In [13]:
import itertools
a = list(Biblical_Hebrew_dic.values())
Xa=list(itertools.chain.from_iterable(a))
print(Xa)
b= list(Period_dic.values())
Xb=list(itertools.chain.from_iterable(b))
print(Xb)

['Genesis', 'Exodus', 'Leviticus', 'Numbers', 'Deuteronomy', 'Joshua', 'Judges', 'Samuel_1', 'Samuel_2', 'Kings_1', 'Kings_2', 'Isaiah', 'Jeremiah', 'Ezekiel', 'Ezra', 'Nehemiah', 'Esther', 'Daniel', 'Chronicles_1', 'Chronicles_2', 'Haggai', 'Zechariah', 'Malachi', 'Song_of_Songs', 'Ecclesiastes']
['Hosea', 'Nahum', 'Zephaniah', 'Habakkuk', 'Isaiah', 'Judges', 'Samuel_1', 'Samuel_2', 'Kings_1', 'Kings_2', 'Micah', 'Proverbs', 'Deuteronomy', 'Psalms', 'Amos', 'Joshua', 'Obadiah', 'Lamentations', 'Ezekiel', 'Jeremiah', 'Genesis', 'Exodus', 'Leviticus', 'Numbers', 'Ruth', 'Haggai', 'Zechariah', 'Malachi', 'Chronicles_1', 'Chronicles_2', 'Ezra', 'Nehemiah', 'Joel', 'Job', 'Ecclesiastes', 'Song_of_Songs', 'Jonah', 'Esther', 'Daniel']


In [14]:
print("not in Biblical_Hebrew_dic")
for name in valid_df.name.unique():
  if(name not in Xa):
    print(name)
print("--------------------------------") 
print("not in Period_dic")
for name in valid_df.name.unique():
  if(name not in Xb):
    print(name)

not in Biblical_Hebrew_dic
Ruth
Habakkuk
Jonah
Obadiah
Joel
Hosea
Micah
Lamentations
Proverbs
Job
Zephaniah
Psalms
Nahum
Amos
--------------------------------
not in Period_dic


In [15]:
Classical_Biblical_Hebrew_df=valid_df[valid_df["Biblical_Hebrew_era"]=='Classical Biblical Hebrew']
Transitional_period_Hebrew_df=valid_df[valid_df["Biblical_Hebrew_era"]=='Transitional period between the classical language and the later language']
Late_Biblical_Hebrew_df=valid_df[valid_df["Biblical_Hebrew_era"]=='Late Biblical Hebrew']
print(Classical_Biblical_Hebrew_df)
print(Transitional_period_Hebrew_df.shape)
print(Late_Biblical_Hebrew_df.shape)
print(valid_df.shape)

     Unnamed: 0    name  verse_idx  \
0           294  Isaiah      294.0   
1          1252  Isaiah     1252.0   
2           279  Isaiah      279.0   
3           541  Isaiah      541.0   
4           619  Isaiah      619.0   
..          ...     ...        ...   
531       22964  Exodus      955.0   
532       22263  Exodus      254.0   
533       23156  Exodus     1147.0   
534       22519  Exodus      510.0   
535       23207  Exodus     1198.0   

                                                 verse               Period  \
0    הכינו לבניו מטבח בעון אבותם בל יקמו וירשו ארץ ...            Monarchic   
1    ואתם עזבי יהוה השכחים את הר קדשי הערכים לגד של...            Monarchic   
2    מכה עמים בעברה מכת בלתי סרה רדה באף גוים מרדף ...            Monarchic   
3    וחניתי כדור עליך וצרתי עליך מצב והקימתי עליך מצרת            Monarchic   
4    עד יערה עלינו רוח ממרום והיה מדבר לכרמל וכרמל ...            Monarchic   
..                                                 ...               

In [16]:
Classical_Biblical_Hebrew_df.to_csv("/content/drive/Shareddrives/Embible/Data/Classical_Biblical_Hebrew_valid_df_not_final.csv", sep='\t', encoding='utf-8',index=False)
Transitional_period_Hebrew_df.to_csv("/content/drive/Shareddrives/Embible/Data/Transitional_period_Hebrew_valid_df_not_final.csv", sep='\t', encoding='utf-8',index=False)
Late_Biblical_Hebrew_df.to_csv("/content/drive/Shareddrives/Embible/Data/Late_Biblical_Hebrew_valid_df_not_final.csv", sep='\t', encoding='utf-8',index=False)

In [17]:
newdf=pd.read_csv("/content/drive/Shareddrives/Embible/Data/Classical_Biblical_Hebrew_valid_df_not_final.csv", sep='\t', encoding='utf-8')
newdf

,Unnamed: 0,name,verse_idx,verse,Period,Biblical_Hebrew_era
0,294,Isaiah,294.0,הכינו לבניו מטבח בעון אבותם בל יקמו וירשו ארץ ...,Monarchic,Classical Biblical Hebrew
1,1252,Isaiah,1252.0,ואתם עזבי יהוה השכחים את הר קדשי הערכים לגד של...,Monarchic,Classical Biblical Hebrew
2,279,Isaiah,279.0,מכה עמים בעברה מכת בלתי סרה רדה באף גוים מרדף ...,Monarchic,Classical Biblical Hebrew
3,541,Isaiah,541.0,וחניתי כדור עליך וצרתי עליך מצב והקימתי עליך מצרת,Monarchic,Classical Biblical Hebrew
4,619,Isaiah,619.0,עד יערה עלינו רוח ממרום והיה מדבר לכרמל וכרמל ...,Monarchic,Classical Biblical Hebrew
...,...,...,...,...,...,...
236,22964,Exodus,955.0,ויאמר אליו אם אין פניך הלכים אל תעלנו מזה,Post-exilic Persian,Classical Biblical Hebrew
237,22263,Exodus,254.0,ויאמר אלהם יהי כן יהוה עמכם כאשר אשלח אתכם ואת...,Post-exilic Persian,Classical Biblical Hebrew
238,23156,Exodus,1147.0,ויעשו שתי משבצת זהב ושתי טבעת זהב ויתנו את שתי...,Post-exilic Persian,Classical Biblical Hebrew
239,22519,Exodus,510.0,ויוצא משה את העם לקראת האלהים מן המחנה ויתיצבו...,Post-exilic Persian,Classical Biblical Hebrew
